In [63]:
# The goal of this Jupyter Notebook is to use an LLM for text classification, i.e., for support cases. The idea is to 
# categorize the support cases and additionally provide a reason for the categorization and also a possible solution 
# to help and relieve the support team.

# See also for advantages and disadvantages of using an LLM to classify text:
# https://sarah-packowski.medium.com/when-and-why-would-you-use-an-llm-for-text-classification-94b39ddc2947

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

# We use Llama.cpp for running an LLM locally:
# https://python.langchain.com/docs/guides/local_llms
# and a model from Mistral AI https://mistral.ai/
# Reason: Mistral AI models are open source and are ranked quite good on this leaderboard:
# https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard
# See also: https://www.reddit.com/r/LocalLLaMA/comments/18hh3qm/best_local_llm_for_german/
# https://mistral.ai/news/mixtral-of-experts/ (too slow on my laptop but better than Mistral-7B)
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
# https://huggingface.co/TheBloke (LLM: quantisation, fine tuning)
# https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF (we need the new GGUF format)
llm = LlamaCpp(
    model_path=R"C:\Users\WernerGaisbauer\LLMs\mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

#llm("The first man on the moon was ... Let's think step by step")

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from C:\Users\WernerGaisbauer\LLMs\mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loade

In [64]:
# For testing, we use a bunch of German support cases.
# https://www.kaggle.com/datasets/jordanrich/german-emails-in-xml
import pandas as pd

df = pd.read_csv("archive/German_emails.csv")
print(df.shape)
print(df.columns.tolist())

(631, 4)
['id', 'category', 'text', 'relevantText']


In [65]:
# Loop over the DataFrame's rows.
for index, row in df.iterrows():
    # Access each column by its name, e.g., row['name'], row['age'], row['city']
    print(f"id: {row['id']}, category: {row['category']}, text: {row['text']}, relevantText: {row['relevantText']}")

id: 1, category: 86, text: 
Hallo,

ich habe seit 2 Tagen in meiner Warehouse Software 
unter Exportieren-File
Optionen ein fast nicht lesbare Schrift. Wie kann ich diese wieder
aendern.

    
        , relevantText: ich habe seit 2 Tagen in meiner Warehouse Software 
unter Exportieren-File
Optionen ein fast nicht lesbare Schrift.
id: 2, category: 86, text: 
Ich habe alle Versionen Sales firstclass von 2006 bis heute. Am Samstag
bekam
die Version WAREHOUSE Sales firstclass 20 Neo, musste nach der Installation
feststellen, dass die Beschriftung der Knoepfe im Exportbildschirm
nicht lesbar sind (z.B. Dateiformat, Navigation, Komprimierung, Schrift,
Sortieren
u.s.w. die komplette Beschriftung unter File-Optionen, Titelseite gestalten
und
Knoepfe gestalten. Nach einer Deinstallation und einer erneuten
Installation hat sich die Darstellung im Exportbildschirm nicht
geaendert.
Da ich Warehouse Schulungen fuer Firmenchefs durchfuehre, ist ein
ordnungsgemaesses Arbeiten nicht moeglich.  

    

In [66]:
# Convert rows to a dictionary.
n = 1
row1_dict = df.iloc[n].to_dict()
#print(row1_dict)
support_email1 = row1_dict["relevantText"]
print(n)
print(support_email1)

n = 6
row6_dict = df.iloc[n].to_dict()
#print(row6_dict)
support_email2 = row6_dict["relevantText"]
print(n)
print(support_email2)

1
Am Samstag
bekam
die Version WAREHOUSE Sales firstclass 20 Neo, musste nach der Installation
feststellen, dass die Beschriftung der Knoepfe im Exportbildschirm
nicht lesbar sind (z.B. Dateiformat, Navigation, Komprimierung, Schrift,
Sortieren
u.s.w. die komplette Beschriftung unter File-Optionen, Titelseite gestalten
und
Knoepfe gestalten.
6
leider kann ich ab heute nicht mehr meine Sammlungen verwalten.
Es tut sich bei Auswahl des entsprechenden Buttons gar nichts mehr.


In [67]:
# https://learn.deeplearning.ai/langchain/lesson/2/models,-prompts-and-parsers
# See section Parse the LLM output string into a Python dictionary.
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [68]:
kategorien = """Data Warehouse, Technischer Support, Rechnungsprobleme, Kontoverwaltung, Feedback, Andere Anfragen"""

In [69]:
kategorie_schema = ResponseSchema(name="kategorie", description=f"Ordne die Anfrage einer der vorgegebenen Kategorien"
                                                                f" zu: {kategorien}.")
begruendung_schema = ResponseSchema(name="begruendung", description="Gib für die Zuordnung eine kurze Begründung an, "
                                                                    "um die Entscheidung zu erklären.")
loesungsvorschlag_schema = ResponseSchema(name="loesungsvorschlag", description="Gib einen Lösungsvorschlag für die "
                                                                                "Support-Anfrage an.")

response_schemas = [kategorie_schema, 
                    begruendung_schema,
                    loesungsvorschlag_schema]

In [70]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [71]:
format_instructions = output_parser.get_format_instructions()

In [72]:
print(format_instructions)
# Unfortunately, the instructions are in English, but we need German instructions, therefore we build the 
# instructions manually below.
# It seems that there is no localisation out of the box available:
# https://github.com/langchain-ai/langchain/issues/5203

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"kategorie": string  // Ordne die Anfrage einer der vorgegebenen Kategorien zu: Data Warehouse, Technischer Support, Rechnungsprobleme, Kontoverwaltung, Feedback, Andere Anfragen.
	"begruendung": string  // Gib für die Zuordnung eine kurze Begründung an, um die Entscheidung zu erklären.
	"loesungsvorschlag": string  // Gib einen Lösungsvorschlag für die Support-Anfrage an.
}
```


In [73]:
from langchain.prompts import ChatPromptTemplate

In [74]:
format_instructions = f"""Die Ausgabe sollte ein Markdown-Code-Snippet sein, formatiert nach dem folgenden Schema, 
einschließlich der führenden und abschließenden "\`\`\`json" und "\`\`\`":

```json
{{
	"kategorie": string  // Ordne die Anfrage einer der vorgegebenen Kategorien zu: {kategorien}.
	"begruendung": string  // Gib für die Zuordnung eine kurze Begründung an, um die Entscheidung zu erklären.
	"loesungsvorschlag": string  // Gib einen Lösungsvorschlag für die Support-Anfrage an.
}}
```"""

In [75]:
template_string = """\
Bitte lies dir die folgende Support-Anfrage sorgfältig durch und erstelle folgende Informationen:

kategorie: Ordne die Anfrage einer der vorgegebenen Kategorien zu: {kategorien}.
begruendung: Gib für die Zuordnung eine kurze Begründung an, um die Entscheidung zu erklären.
loesungsvorschlag: Gib einen Lösungsvorschlag für die Support-Anfrage an.

Support-Anfrage: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_string)

messages = prompt.format_messages(text=support_email1,
                                  format_instructions=format_instructions,
                                  kategorien=kategorien)

In [76]:
print(messages[0].content)

Bitte lies dir die folgende Support-Anfrage sorgfältig durch und erstelle folgende Informationen:

kategorie: Ordne die Anfrage einer der vorgegebenen Kategorien zu: Data Warehouse, Technischer Support, Rechnungsprobleme, Kontoverwaltung, Feedback, Andere Anfragen.
begruendung: Gib für die Zuordnung eine kurze Begründung an, um die Entscheidung zu erklären.
loesungsvorschlag: Gib einen Lösungsvorschlag für die Support-Anfrage an.

Support-Anfrage: Am Samstag
bekam
die Version WAREHOUSE Sales firstclass 20 Neo, musste nach der Installation
feststellen, dass die Beschriftung der Knoepfe im Exportbildschirm
nicht lesbar sind (z.B. Dateiformat, Navigation, Komprimierung, Schrift,
Sortieren
u.s.w. die komplette Beschriftung unter File-Optionen, Titelseite gestalten
und
Knoepfe gestalten.

Die Ausgabe sollte ein Markdown-Code-Snippet sein, formatiert nach dem folgenden Schema, 
einschließlich der führenden und abschließenden "\`\`\`json" und "\`\`\`":

```json
{
	"kategorie": string  // Ordn

In [77]:
response = llm(messages[0].content)

kategorie: Andere Anfragen
begruendung: Die Anfrage behandelt eine Nichtlesbarkeit von Beschriftungen in einem Exportbildschirm. Diese Frage gehört daher nicht zu den oben aufgeführten Kategorien und wird als "Andere Anfragen" eingeordnet.
loesungsvorschlag: Die Nichtlesbarkeit der Beschriftungen in einem Exportbildschirm kann durch eine Verbesserung der Schriftgröße und -qualität im Exportbildschirm gelöst werden. Darüber hinaus können die Beschriftungen auch direkt im Quellcode bearbeitet werden, damit sie lesbar sind, ohne dass die Exportschritte umgestellt werden müssen.


llama_print_timings:        load time =   87756.20 ms
llama_print_timings:      sample time =      63.12 ms /   173 runs   (    0.36 ms per token,  2740.90 tokens per second)
llama_print_timings: prompt eval time =   87753.15 ms /   462 tokens (  189.94 ms per token,     5.26 tokens per second)
llama_print_timings:        eval time =   55694.23 ms /   172 runs   (  323.80 ms per token,     3.09 tokens per second)
llama_print_timings:       total time =  144848.42 ms /   634 tokens


In [78]:
print(response)


kategorie: Andere Anfragen
begruendung: Die Anfrage behandelt eine Nichtlesbarkeit von Beschriftungen in einem Exportbildschirm. Diese Frage gehört daher nicht zu den oben aufgeführten Kategorien und wird als "Andere Anfragen" eingeordnet.
loesungsvorschlag: Die Nichtlesbarkeit der Beschriftungen in einem Exportbildschirm kann durch eine Verbesserung der Schriftgröße und -qualität im Exportbildschirm gelöst werden. Darüber hinaus können die Beschriftungen auch direkt im Quellcode bearbeitet werden, damit sie lesbar sind, ohne dass die Exportschritte umgestellt werden müssen.


In [79]:
output_dict = output_parser.parse(response)

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

In [80]:
output_dict

NameError: name 'output_dict' is not defined

In [81]:
type(output_dict)

NameError: name 'output_dict' is not defined

In [82]:
output_dict.get('kategorie')

NameError: name 'output_dict' is not defined